In [206]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
import re

In [207]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

currency_symbols = ["€", "$", "£"]

def transfer_markt_soup(season_year=2020):
    url = "https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/"
    payload = {'saison_id': season_year}

    page_req = requests.get(url, headers=headers, params=payload)
    print(page_req.url)

    return BeautifulSoup(page_req.content, "html.parser")

In [433]:
season_year = 2007
page_soup = transfer_markt_soup(season_year)

https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2007


## Club Main Data: Foreigners, Market Value, and Squad Size

In [434]:
table_div = page_soup.find(id='yw1')
table_main = table_div.find('table', class_="items")

In [435]:
table_head = []
for i in table_main.find_all("th"):
 title = i.text
 table_head.append(title)

In [436]:
table_head

['Club',
 'name',
 'Squad',
 'ø age',
 'Foreigners',
 'ø market value',
 'Total market value']

In [437]:
table_body = []
rows = table_main.find("tbody").find_all("tr")
for row in rows:
    cells = row.find_all("td")
    row_data = []
    for i in range(len(cells)):
        cell_value_raw = cells[i].get_text()
        try:
            cell_value_final = float(cell_value_raw)
        except ValueError:
            if cell_value_raw != "":
                if any(cell_value_raw[0] == el for el in currency_symbols):
                    cell_value_raw = cell_value_raw[1:-1]
                    cell_value_final = float(cell_value_raw) 
                else:
                    cell_value_final = cell_value_raw.strip()
            else:
                cell_value_final = cell_value_raw
        row_data.append(cell_value_final)
    table_body.append(row_data)

In [438]:
# Making pandas dataframe to clean up data 
main_df = pd.DataFrame(data=table_body, columns=table_head)
main_df = main_df.drop('Club',axis=1)
main_df

,name,Squad,ø age,Foreigners,ø market value,Total market value
0,FC Internazionale,48.0,25.4,31.0,6.46,310.00
1,AC Milan,41.0,27.1,18.0,6.81,279.40
2,AS Roma,37.0,25.7,19.0,6.61,244.40
3,Juventus FC,38.0,25.3,14.0,6.23,236.85
4,ACF Fiorentina,46.0,23.3,16.0,3.73,171.35
5,Udinese Calcio,49.0,22.7,24.0,2.35,115.05
6,SS Lazio,40.0,26.1,15.0,2.77,110.80
7,US Palermo,43.0,23.7,6.0,2.44,104.80
8,SSC Napoli,30.0,26.8,9.0,2.57,76.95
9,UC Sampdoria,41.0,24.3,8.0,1.85,75.70


## Club Ranking General Data

In [439]:
table_div = page_soup.find(id='yw4')
table_main = table_div.find('table', class_="items")

In [440]:
table_head = []
for i in table_main.find_all("th"):
 title = i.text
 if title == '\xa0':
    title = 'MP'
 table_head.append(title)

In [441]:
table_head

['#', 'Club', 'MP', '+/-', 'Pts']

In [442]:
table_body = []
rows = table_main.find("tbody").find_all("tr")
for row in rows:
    cells = row.find_all("td")
    row_data = []
    for i in range(len(cells)):
        cell_value_raw = cells[i].get_text().strip()
        if re.match(r"^[0-9]+$", cell_value_raw) and cell_value_raw != "":
            cell_value_final = int(cell_value_raw)
        elif cell_value_raw == "":
            continue
        else:
            cell_value_final = cell_value_raw
        row_data.append(cell_value_final)
    table_body.append(row_data)

In [443]:
df2 = pd.DataFrame(data=table_body, columns=table_head)
df2

,#,Club,MP,+/-,Pts
0,1,Inter,38,43,85
1,2,AS Roma,38,35,82
2,3,Juventus,38,35,72
3,4,Fiorentina,38,16,66
4,5,AC Milan,38,28,64
5,6,Sampdoria,38,10,60
6,7,Udinese,38,-5,57
7,8,SSC Napoli,38,-3,50
8,9,Atalanta BC,38,-4,48
9,10,Genoa,38,-8,48


## Merging the two datasets

In [444]:
# Fuzzy Matching team names in preparation of the merge
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [445]:
left_main_df = fuzzy_merge(main_df, df2, 'name', 'Club', threshold=80)
right_df2 = df2

In [446]:
df3 = left_main_df.merge(right_df2, left_on='matches', right_on='Club')
df3 = df3.drop(columns=['matches', 'Club'])
df3 = df3.rename(columns={"+/-": "GD", "#": "Ranking", "ø market value": "Average market value", "ø age": "Average age"})
df3 = df3.astype({'GD': 'int64'})
df3['Season'] = season_year
df3

,name,Squad,Average age,Foreigners,Average market value,Total market value,Ranking,MP,GD,Pts,Season
0,FC Internazionale,48.0,25.4,31.0,6.46,310.00,1,38,43,85,2007
1,AC Milan,41.0,27.1,18.0,6.81,279.40,5,38,28,64,2007
2,AS Roma,37.0,25.7,19.0,6.61,244.40,2,38,35,82,2007
3,Juventus FC,38.0,25.3,14.0,6.23,236.85,3,38,35,72,2007
4,ACF Fiorentina,46.0,23.3,16.0,3.73,171.35,4,38,16,66,2007
5,Udinese Calcio,49.0,22.7,24.0,2.35,115.05,7,38,-5,57,2007
6,SS Lazio,40.0,26.1,15.0,2.77,110.80,12,38,-4,46,2007
7,US Palermo,43.0,23.7,6.0,2.44,104.80,11,38,-10,47,2007
8,SSC Napoli,30.0,26.8,9.0,2.57,76.95,8,38,-3,50,2007
9,UC Sampdoria,41.0,24.3,8.0,1.85,75.70,6,38,10,60,2007


In [447]:
os.makedirs("data", exist_ok=True)  
df3.to_csv(f"data/SerieA_{season_year}.csv")  